
### Objective: 

In this assignment, implement the Node2Vec algorithm, a random-walk-based GNN, to learn node embeddings. Train a classifier using the learned embeddings to predict node labels.

### Dataset: 

Cora dataset: The dataset consists of 2,708 nodes (scientific publications) with 5,429 edges (citations between publications). Each node has a feature vector of size 1,433, and there are 7 classes (research topics).
Skeleton Code:

In [2]:
!pip install node2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 18.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 5.4 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 3.8 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 7.2 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.1
    Uninstalling tqdm-4.64.1:
      Successfully uninstalled tqdm-4.64.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: networkx
    Found existing installation: networkx 2.8.4
    Uninstalling networkx-2.8.4:
      Successfully uninstalled networkx-2.8.4
  Attempting uninst

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx
from node2vec import Node2Vec  # Importing Node2Vec for the random walk

# Load the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora')

# Prepare data
data = dataset[0]

# Convert to networkx for random walk
import networkx as nx
G = to_networkx(data, to_undirected=True)

# Node2Vec configuration
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=2) 
model = node2vec.fit(window=10, min_count=1)

# Embeddings for each node
embeddings = model.wv  # Node embeddings

# Define a simple classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Initialize classifier and optimizer
classifier = Classifier(64, 7)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(100):
    classifier.train()
    optimizer.zero_grad()
    
    # Get node embeddings as input
    output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))
    
    loss = criterion(output, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")


Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 100/100 [00:29<00:00,  3.40it/s]
/var/folders/xy/dc9cct710_3fw4sk4kwnptnm0000gn/T/ipykernel_6643/523172613.py:45: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:248.)
  output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))


Epoch 0, Loss: 2.031139850616455
Epoch 10, Loss: 1.2861847877502441
Epoch 20, Loss: 0.9358958601951599
Epoch 30, Loss: 0.7734335064888
Epoch 40, Loss: 0.6942480206489563
Epoch 50, Loss: 0.649330198764801
Epoch 60, Loss: 0.6194333434104919
Epoch 70, Loss: 0.5978078842163086
Epoch 80, Loss: 0.581070065498352
Epoch 90, Loss: 0.5675488114356995
Training complete!


## Explanation:
Node2Vec generates node embeddings by simulating random walks on the graph. These walks capture structural properties of nodes.
The generated embeddings are then used to train a classifier for predicting node labels.
The Cora dataset is a benchmark graph where nodes are papers and edges are citations.

## Questions (1 point each):
1. What would happen if we increased the number of walks (num_walks) per node? How might this affect the learned embeddings?
**Ans:** Increasing the number of walks per node will increase the amount of information gathered about each node’s neighborhood. As a result, the embeddings can better capture both the local and global structure of the graph, potentially improving their quality. The learned embeddings might become more stable and consistent, leading to better generalization. However, this increases the computational cost of running the classifier and could potentially lead to overfitting. (See code below).



2. What would happen if we reduced the walk length (walk_length)? How would this influence the structural information captured by the embeddings?
**Ans:** Shorter walks lead to more localized information. As a result, the embeddings will reflect local structural properties but may not capture overall structure or global relationships within the graph. Reducing the walk length decreases the number of steps taken in each random walk, which will reduce the computational time needed to generate the walks and train the embeddings. However, this may lead to poorer performance especially for tasks that require a more global understanding of the structure of the data.


3. What would happen if we used directed edges instead of undirected edges for the random walks?
**Ans:** Directed edges reflect the asymmetric relationships between nodes (e.g., in a citation network, paper A cites paper B, but B may not cite A). This allows the embeddings to capture the directionality of these relationships, leading to more detailed representations that account for the flow of information or relationships in the graph. Additionally, since directed edges only allow walks in certain directions, this might change the nieghbors encountered thus leading to differing embeddings. For example, if paper A cites paper B but not the other way around, A and B will have distinct representations as opposed to an undirected edge where they will have more similar representations. Random walks might also get trapped in certain areas of the graph because of directionality.


4. What would happen if we added more features to the nodes (e.g., 2000-dimensional features instead of 1433)?
**Ans:** Adding more features increases the potential for richer and more expressive node embeddings since each node's representation can contain more information. This can also lead to improved performance since the model has more input data to capture complex semantic relationships. However, this also leads to increases memory usage and computational cost, especially for large graphs, since each node contains more information. Moreover, the additional features might be redundant and cause overfitting.


5. What would happen if we used a different dataset with more classes? Would the classifier performance change significantly?
**Ans:** If we used a different dataset with more classes, it would become harder to distinguish between various classes for each node. The classifier will need to learn more complex decision boundaries potentially leading to lower overall accuracy. This also creates the need for more expressive embeddings to distinguish various node classes. If the new dataset has imbalanced class distributions, the classifier might struggle with performance on the less frequent classes. The risk of overfitting also increases, introducing the need for some type of regularization (L1, L2, or dropout).


6. What would happen if we used a larger embedding dimension (e.g., 128 instead of 64)? How would this affect the model’s performance and training time?
**Ans:** A larger embedding dimension gives the model more representational capacity. This means that the embeddings can capture more detailed structural information and node attributes, allowing for more expressive node representations. In some cases, this can also lead to improved performance because the model has more information to distinguish between various node classes. For example, if two nodes are structurally similar but belong to different classes, increasing the embedding dimension might allow the classifier to absorb more information about the node's intricate differences. However, it can also lead to the curse of dimensionality which makes it harder for the model to generalize and find meaningful patterns in the data. It will also lead to increased training time and memory usage because (in this case) we are doubling the size of each node's embedding vector. (See code below).



### Extra credit: 
1. What would happen if we increased the window size (window) for the skip-gram model? How would it affect the embedding quality?
**Ans:** A wider window size allows the model to consider a broader neighborhood around each node during training. Embeddings will incorporate information from nodes that are farther away, which can help capture more global structural properties of the graph. By considering a larger context, the embeddings can generalize better across the graph, capturing patterns that span beyond immediate neighbors. However, there is a risk that the embeddings for all nodes in a graph will become overly similar, especially in densely connected regions, leading to over-smoothing and poorer localized performance.


## No points, just for you to think about
7. What would happen if we removed self-loops from the graph before training Node2Vec?

9. What would happen if we applied normalization to the node embeddings before feeding them to the classifier?

#### Question 1 - Increasing num_walks

In [2]:
# Node2Vec configuration
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=500, workers=2) 
model = node2vec.fit(window=10, min_count=1)

# Embeddings for each node
embeddings = model.wv  # Node embeddings

# Define a simple classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Initialize classifier and optimizer
classifier = Classifier(64, 7)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(100):
    classifier.train()
    optimizer.zero_grad()
    
    # Get node embeddings as input
    output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))
    
    loss = criterion(output, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")

Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 2): 100%|██████████| 250/250 [01:08<00:00,  3.64it/s]


Epoch 0, Loss: 2.0590436458587646
Epoch 10, Loss: 1.3194230794906616
Epoch 20, Loss: 0.9471554756164551
Epoch 30, Loss: 0.7831547260284424
Epoch 40, Loss: 0.702115535736084
Epoch 50, Loss: 0.6545844078063965
Epoch 60, Loss: 0.6221349835395813
Epoch 70, Loss: 0.5981424450874329
Epoch 80, Loss: 0.5794676542282104
Epoch 90, Loss: 0.5643690824508667
Training complete!


#### Question 6 - Increased embedding dimension

In [4]:
# Node2Vec configuration
node2vec = Node2Vec(G, dimensions=128, walk_length=30, num_walks=500, workers=2) 
model = node2vec.fit(window=10, min_count=1)

# Embeddings for each node
embeddings = model.wv  # Node embeddings

# Define a simple classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Initialize classifier and optimizer
classifier = Classifier(128, 7)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(100):
    classifier.train()
    optimizer.zero_grad()
    
    # Get node embeddings as input
    output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))
    
    loss = criterion(output, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")

Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 250/250 [01:07<00:00,  3.72it/s]


Epoch 0, Loss: 2.025282382965088
Epoch 10, Loss: 1.1157904863357544
Epoch 20, Loss: 0.7768505811691284
Epoch 30, Loss: 0.6613138914108276
Epoch 40, Loss: 0.6017512679100037
Epoch 50, Loss: 0.5619878768920898
Epoch 60, Loss: 0.532753586769104
Epoch 70, Loss: 0.5096790790557861
Epoch 80, Loss: 0.4906686544418335
Epoch 90, Loss: 0.4746033251285553
Training complete!
